In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import datasets, transforms
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,matthews_corrcoef,f1_score

USE_CUDA = True

In [ ]:
import copy

In [ ]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, out_channels=256, kernel_size=9):
        super(ConvLayer, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=kernel_size,
                               stride=1
                             )

    def forward(self, x):
        return F.relu(self.conv(x))

In [ ]:
class PrimaryCaps(nn.Module):
    def __init__(self, num_capsules=8, in_channels=256, out_channels=8, kernel_size=9):
        super(PrimaryCaps, self).__init__()

        self.capsules = nn.ModuleList([
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=2, padding=0)
                          for _ in range(num_capsules)])

    def forward(self, x):
        u = [capsule(x) for capsule in self.capsules]
        u = torch.stack(u, dim=1)
        u = u.view(x.size(0), 8*24*24 , -1)
        return self.squash(u)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

In [ ]:
class DigitCaps(nn.Module):
    def __init__(self, num_capsules=2, num_routes=8*24*24, in_channels=8, out_channels=32):
        super(DigitCaps, self).__init__()

        self.in_channels = in_channels
        self.num_routes = num_routes
        self.num_capsules = num_capsules

        self.W = nn.Parameter(torch.randn(1, num_routes, num_capsules, out_channels, in_channels))

    def forward(self, x):
        batch_size = x.size(0)
        x = torch.stack([x] * self.num_capsules, dim=2).unsqueeze(4)
        W = torch.cat([self.W] * batch_size, dim=0)
        u_hat = torch.matmul(W, x)

        b_ij = Variable(torch.zeros(1, self.num_routes, self.num_capsules, 1))
        if USE_CUDA:
            b_ij = b_ij.cuda()

        num_iterations = 3
        for iteration in range(num_iterations):
            c_ij = F.softmax(b_ij)
            c_ij = torch.cat([c_ij] * batch_size, dim=0).unsqueeze(4)

            s_j = (c_ij * u_hat).sum(dim=1, keepdim=True)
            v_j = self.squash(s_j)

            if iteration < num_iterations - 1:
                a_ij = torch.matmul(u_hat.transpose(3, 4), torch.cat([v_j] * self.num_routes, dim=1))
                b_ij = b_ij + a_ij.squeeze(4).mean(dim=0, keepdim=True)

        return v_j.squeeze(1)

    def squash(self, input_tensor):
        squared_norm = (input_tensor ** 2).sum(-1, keepdim=True)
        output_tensor = squared_norm *  input_tensor / ((1. + squared_norm) * torch.sqrt(squared_norm))
        return output_tensor

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.reconstraction_layers = nn.Sequential(
            nn.Linear(32 * 2, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 1*64*64),
            nn.Sigmoid()
        )

    def forward(self, x, data):
        classes = torch.sqrt((x ** 2).sum(2))
        classes = F.softmax(classes,dim=1)

        _, max_length_indices = classes.max(dim=1)
        masked = Variable(torch.sparse.torch.eye(2))
        if USE_CUDA:
            masked = masked.cuda()
        masked = masked.index_select(dim=0, index=max_length_indices.squeeze(1).data)

        reconstructions = self.reconstraction_layers((x * masked[:, :, None, None]).view(x.size(0), -1))
        reconstructions = reconstructions.view(-1, 1, 64, 64)

        return reconstructions, masked

In [ ]:
class CapsNet(nn.Module):
    def __init__(self,Primary_capsule_num=8):
        super(CapsNet, self).__init__()
        self.conv_layer = ConvLayer()
        self.primary_capsules = PrimaryCaps(num_capsules=Primary_capsule_num)
        self.digit_capsules = DigitCaps(in_channels=Primary_capsule_num)
        self.decoder = Decoder()
        self.mse_loss = nn.MSELoss()

    def forward(self, data):
        output = self.digit_capsules(self.primary_capsules(self.conv_layer(data)))
        reconstructions, masked = self.decoder(output, data)
        print(output.size())
        return output, reconstructions, masked

    def loss(self, data, x, target, reconstructions):
        return self.margin_loss(x, target) + self.reconstruction_loss(data, reconstructions)

    def margin_loss(self, x, labels, size_average=True):
        batch_size = x.size(0)

        v_c = torch.sqrt((x**2).sum(dim=2, keepdim=True))

        left = F.relu(0.9 - v_c).view(batch_size, -1)
        right = F.relu(v_c - 0.1).view(batch_size, -1)
        loss = labels * left + 0.5 * (1.0 - labels) * right
        loss = loss.sum(dim=1).mean()

        return loss

    def reconstruction_loss(self, data, reconstructions):
        loss = self.mse_loss(reconstructions.view(reconstructions.size(0), -1), data.view(reconstructions.size(0), -1))
        return loss * 0.0005

In [ ]:
import numpy as np
import pandas as pd
batch_size = 64
n_epochs = 20
res = 64

fig_test = pd.read_csv("test.txt")
y_test=fig_test['label']
y_test
str_list_test=[]
for i in fig_test['figure']:
  temp=np.array(i.split(" "),dtype=np.float32).reshape(res,res)
  str_list_test.append(temp)


In [ ]:
X_test=torch.tensor(str_list_test)
X_test=X_test.unsqueeze(1)

<ipython-input-10-39fd547b6980>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  X_test=torch.tensor(str_list_test)


In [ ]:

from torch.utils.data import DataLoader, TensorDataset
test_dataset=TensorDataset(X_test,torch.tensor(y_test))
test_loader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# X.size()

In [ ]:
# num_cap_list=[8]

In [ ]:
model_cap = torch.load('capsule_net.pth').to("cuda")

test_dataset = TensorDataset(X_test,torch.tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
with torch.no_grad():
    all_pred = []
    all_score = []
    all_prob=[]
    all_type=[]
    for step, (batch_x, batch_y) in enumerate(test_loader):
        output, reconstructions, masked = model_cap(batch_x.to("cuda"))
        pred_labels = np.argmax(masked.data.cpu().numpy(), 1)
        pred  = np.argmax(masked.cpu().detach().numpy(),axis=1).tolist()
        classes = torch.sqrt((output ** 2).sum(2))
        classes = F.softmax(classes,dim=1).detach().cpu().numpy().tolist()
        all_prob += classes
        all_pred += pred
        all_type += output.reshape(-1,64)



    tn, fp, fn, tp = confusion_matrix(y_test, all_pred).ravel()
    perftab = {"CM": confusion_matrix(y_test, all_pred),
            'ACC': (tp + tn) / (tp + fp + fn + tn),
            'SEN': tp / (tp + fn),
            'PREC': tp / (tp + fp),
            "SPEC": tn / (tn + fp),
            "MCC": matthews_corrcoef(y_test, all_pred),
            "F1": f1_score(y_test, all_pred)
    }



<ipython-input-15-219208592d22>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_cap = torch.load('capsule_net.pth').to("cuda")


torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([64, 2, 32, 1])
torch.Size([18, 2, 32, 1])


In [ ]:
perftab

{'CM': array([[1043,   54],
        [  20, 1077]]),
 'ACC': 0.9662716499544212,
 'SEN': 0.9817684594348223,
 'PREC': 0.9522546419098143,
 'SPEC': 0.95077484047402,
 'MCC': 0.9329915254664284,
 'F1': 0.9667863554757631}

In [ ]:
# score_result=score_result.squeeze()